# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [1]:
import bac.simulate.gromacs as gmx

In [2]:
em_steep = gmx.Run(integrator='steep', number_of_steps=5000)
em_steep.output_name = 'em_steep'
em_steep.coordinates = '/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro'
em_steep.topology = '/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top'

In [3]:
em_steep.coordinates.exists()

True

In [4]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10

In [5]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.non_bonded_controller.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 1
nbc.neighbor_list_type = gmx.non_bonded_controller.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.non_bonded_controller.PeriodicBoundaryConditionType.xyz
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.non_bonded_controller.CoulombType.pme
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_type = gmx.non_bonded_controller.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.non_bonded_controller.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.non_bonded_controller.LongRangeDispersionCorrectionType.energy_and_pressure

In [6]:
fe = gmx.FreeEnergyController()

fe.initial_lambda_state = 0
fe.delta_lambda = 0
fe.calculate_lambda_neighbors = 1
fe.van_der_waals_lambdas = [f'{x*0.05:.2f}' for x in range(21)] 
fe.coulomb_lambdas = [f'{0:.2f}']*21
fe.bonded_lambdas = [f'{0:.2f}']*21
fe.restraint_lambdas = [f'{0:.2f}']*21
fe.mass_lambdas = [f'{0:.2f}']*21
fe.temperature_lambdas = [f'{0:.2f}']*21

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

em_steep.free_energy_controller = fe

In [7]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

Next energy minimization step

In [8]:
em_bfgs = next(em_steep)

In [9]:
em_bfgs.integrator = gmx.Integrator.l_bfgs
em_bfgs.constraints.type = gmx.ConstraintType.none
em_bfgs.name = 'em_bfgs'

In [10]:
nvt = next(em_bfgs)
nvt.output_name = 'nvt'

In [11]:
nvt.integrator = 'sd'
nvt.delta_time = 0.002
nvt.number_of_steps = 50000
nvt.com_motion_removal_frequency = 100

nvt.generate_velocities = True
nvt.generate_temperature = 300

In [12]:
nvt.temperature_controller.groups = 'system'
nvt.temperature_controller.time = 1
nvt.temperature_controller.temperature = 300

## Operation and OperationQueue

In [13]:
from bac.simulate import Workflow
from bac.simulate.run_modifier import ReplicaModifier, LambdaModifier
from itertools import product

In [14]:
wf = Workflow('MPB', name=None)

In [15]:
wf.add_simulation(em_steep)
wf.add_simulation(em_bfgs)
wf.add_simulation(nvt)

In [16]:
wf.modifiers = [ReplicaModifier(count=5), LambdaModifier(lambda_window_count=13)]

In [17]:
wf.preprocess_simulations()

gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdru

gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdru

gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gr

gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gr

gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
gmx grompp -f steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_steep.tpr
gmx mdrun -nt 1 -deffnm em_steep
gmx grompp -f em_bfgs.mdp -c em_steep.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o em_bfgs.tpr
gmx mdrun -nt 1 -deffnm em_bfgs
gmx grompp -f sd.mdp -c em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/original_files/topol.top -o nvt.tpr
gmx mdrun -nt 1 -deffnm nvt
